In [1]:
from torchvision import transforms, datasets
from tqdm import tqdm
from torch.utils.data import DataLoader
import numpy as np
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from time import time

In [2]:
class Unet(nn.Module):
    def __init__(self, num_classes):
        super(Unet, self).__init__()
        self.stage_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        self.stage_2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )
        
        self.stage_3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        
        self.stage_4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        
        self.stage_5 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
        )
        
        self.upsample_4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512,kernel_size=4,stride=2, padding=1) 
        )
        self.upsample_3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256,kernel_size=4,stride=2, padding=1) 
        )
        self.upsample_2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128,kernel_size=4,stride=2, padding=1) 
        )
        self.upsample_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64,kernel_size=4,stride=2, padding=1) 
        )
        
        self.stage_up_4 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        self.stage_up_3 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )    
        
        self.stage_up_2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        ) 
        self.stage_up_1 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )    
        
        self.final = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, padding=1),
        )
        
    def forward(self, x):
        x = x.float()
        #下采样过程
        stage_1 = self.stage_1(x)
        stage_2 = self.stage_2(stage_1)
        stage_3 = self.stage_3(stage_2)
        stage_4 = self.stage_4(stage_3)
        stage_5 = self.stage_5(stage_4)
        
        #1024->512
        up_4 = self.upsample_4(stage_5)
        #512+512 -> 512\
        
        up_4_conv = self.stage_up_4(torch.cat([up_4, stage_4], dim=1))
        
        #512 -> 256
        up_3 = self.upsample_3(up_4_conv)
        #256+256 -> 256
        up_3_conv = self.stage_up_3(torch.cat([up_3, stage_3], dim=1))
        
        up_2 = self.upsample_2(up_3_conv)
        up_2_conv = self.stage_up_2(torch.cat([up_2, stage_2], dim=1))
        
        up_1 = self.upsample_1(up_2_conv)
        up_1_conv = self.stage_up_1(torch.cat([up_1, stage_1], dim=1))   
        
        output = self.final(up_1_conv)
        
        return output

In [3]:
class SegDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, masks_dir):
        # 使用Albumentations定义变换
        self.transforms = A.Compose([
            A.Resize(224, 224),  # 调整图像大小
            A.HorizontalFlip(),  # 随机水平翻转
            A.VerticalFlip(),    # 随机垂直翻转
            A.Normalize(),  # 归一化
            ToTensorV2()  # 转换为张量
        ])
        self.ids = os.listdir(images_dir)
        # self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        # self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # self.images_fps = [os.path.normpath(os.path.join(images_dir, image_id)) for image_id in self.ids]
        # self.masks_fps = [os.path.normpath(os.path.join(masks_dir, image_id)) for image_id in self.ids]
        self.images_fps = [images_dir+"/"+ image_id for image_id in self.ids]
        self.masks_fps = [masks_dir+"/"+ image_id for image_id in self.ids]

    def __getitem__(self, i):
        # 使用cv2读取图像
        image = cv2.imread(self.images_fps[i])
        mask = cv2.imread(self.masks_fps[i].replace(".jpg", ".png"))
        # print("mask",mask)
        # print("image",image)
        # print(self.images_fps[i])
        # print(self.masks_fps[i])

        # 将图像和掩码从BGR转换为RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        
        image = np.array(image)
        mask = np.array(mask)


        # 应用预处理变换
        transformed = self.transforms(image=image, mask=mask)

        # 返回变换后的图像和掩码（假设掩码是单通道的，选择第一个通道）
        return transformed['image'], transformed['mask'][:, :, 0]

    def __len__(self):
        return len(self.ids)

In [4]:
x_train_dir = "NEU_Seg-main/NEU_Seg-main/images/training"
y_train_dir = "NEU_Seg-main/NEU_Seg-main/annotations/training"
x_test_dir = "NEU_Seg-main/NEU_Seg-main/images/test"
y_test_dir = "NEU_Seg-main/NEU_Seg-main/annotations/test"

train_dataset = SegDataset(
    x_train_dir, 
    y_train_dir, 
)
test_dataset = SegDataset(
    x_test_dir, 
    y_test_dir, 
)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True,drop_last=True)

In [6]:
print(train_dataset[0])

(tensor([[[-0.9192, -0.9020, -0.9363,  ..., -0.5767, -0.5938, -0.5596],
         [-0.9192, -0.9020, -0.9363,  ..., -0.6452, -0.6281, -0.5767],
         [-0.9192, -0.9192, -0.9363,  ..., -0.5938, -0.6109, -0.5596],
         ...,
         [-0.8164, -0.8335, -0.8507,  ..., -0.3369, -0.3541, -0.4226],
         [-0.8335, -0.8164, -0.8335,  ..., -0.2856, -0.2856, -0.3198],
         [-0.8678, -0.8507, -0.8507,  ..., -0.3027, -0.2856, -0.2342]],

        [[-0.8102, -0.7927, -0.8277,  ..., -0.4601, -0.4776, -0.4426],
         [-0.8102, -0.7927, -0.8277,  ..., -0.5301, -0.5126, -0.4601],
         [-0.8102, -0.8102, -0.8277,  ..., -0.4776, -0.4951, -0.4426],
         ...,
         [-0.7052, -0.7227, -0.7402,  ..., -0.2150, -0.2325, -0.3025],
         [-0.7227, -0.7052, -0.7227,  ..., -0.1625, -0.1625, -0.1975],
         [-0.7577, -0.7402, -0.7402,  ..., -0.1800, -0.1625, -0.1099]],

        [[-0.5844, -0.5670, -0.6018,  ..., -0.2358, -0.2532, -0.2184],
         [-0.5844, -0.5670, -0.6018,  ..., -

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Unet(num_classes=4).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()  # 定义损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 优化器
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"训练轮数 [{epoch + 1}/{num_epochs}]"):
        images, labels = images.to(device), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    avg_train_loss = running_loss / len(train_loader)

    # 测试模型
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    avg_test_loss = test_loss / len(test_loader)

    print(f"训练集上的loss: {avg_train_loss:.4f} - 训练集上的准确度: {train_accuracy * 100:.2f}%")
    print(f"测试集上的loss: {avg_test_loss:.4f} - 测试集上的准确度: {test_accuracy * 100:.2f}%")

训练轮数 [1/10]: 100%|██████████| 226/226 [02:14<00:00,  1.68it/s]


训练集上的loss: 0.2609 - 训练集上的准确度: 4580485.37%
测试集上的loss: 0.2556 - 测试集上的准确度: 4594756.25%


训练轮数 [2/10]:  61%|██████    | 137/226 [01:24<00:54,  1.63it/s]


KeyboardInterrupt: 

In [8]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """Evaluate accuracy of a model on the given data set using a GPU."""
    if device is None and isinstance(net, nn.Module):
        device = next(iter(net.parameters())).device
    net.eval()  # Set the model to evaluation mode
    metric = [0.0, 0.0]  # Sum of correct predictions, number of predictions
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric[0] += (net(X).argmax(dim=1) == y).float().sum().item()
            metric[1] += y.numel()
    return metric[0] / metric[1]

def train_batch(net, features, labels, loss, trainer, device):
    """Train a batch of data."""
    if isinstance(features, list):
        features = [x.to(device) for x in features]
    else:
        features = features.to(device)
    labels = labels.to(device)
    net.train()
    trainer.zero_grad()
    predictions = net(features)
    l = loss(predictions, labels)
    l.backward()
    trainer.step()
    with torch.no_grad():
        acc = (predictions.argmax(dim=1) == labels).float().mean().item()
    return l.item(), acc

def train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, scheduler, devices=None):
    if devices is None:
        devices = [torch.device('cuda' if torch.cuda.is_available() else 'cpu')]
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    loss_list = []
    train_acc_list = []
    test_acc_list = []
    epochs_list = []
    time_list = []

    for epoch in range(num_epochs):
        metric = [0.0, 0.0, 0.0, 0.0]  # Sum of training loss, sum of training accuracy, no. of examples, no. of predictions
        start_time = time()

        # for i, (features, labels) in enumerate(train_iter):
        for i, (features, labels) in tqdm(enumerate(train_iter), total=len(train_iter), desc=f"训练轮数 [{epoch + 1}/{num_epochs}]"):
            l, acc = train_batch(net, features, labels.long(), loss, trainer, devices[0])################################
            metric[0] += l * labels.shape[0]
            metric[1] += acc * labels.numel()
            metric[2] += labels.shape[0]
            metric[3] += labels.numel()

        test_acc = evaluate_accuracy_gpu(net, test_iter, devices[0])
        scheduler.step()

        epoch_time = time() - start_time
        print(f"Epoch {epoch} \n--- Loss {metric[0] / metric[2]:.3f} \n---  Train Acc {metric[1] / metric[3]:.3f} \n--- Test Acc {test_acc:.3f} \n--- Time {epoch_time:.1f}s")

        # Save training data
        loss_list.append(metric[0] / metric[2])
        train_acc_list.append(metric[1] / metric[3])
        test_acc_list.append(test_acc)
        epochs_list.append(epoch)
        time_list.append(epoch_time)

        df = pd.DataFrame({
            'epoch': epochs_list,
            'loss': loss_list,
            'train_acc': train_acc_list,
            'test_acc': test_acc_list,
            'time': time_list
        })
        df.to_excel("savefile/Unet.xlsx", index=False)

        # Save model checkpoints
        if (epoch + 1) % 5 == 0:
            torch.save(net.state_dict(), f'checkpoints/Unet_{epoch+1}.pth')

# Example usage:
# train_ch13(net, train_loader, test_loader, loss_fn, optimizer, num_epochs, scheduler)


In [9]:
num_epochs = 10
lossf = nn.CrossEntropyLoss(ignore_index=255)
#选用adam优化器来训练
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1, last_epoch=-1)
train_ch13(model, train_loader, test_loader, lossf, optimizer, num_epochs, scheduler)

训练轮数 [1/10]: 100%|██████████| 226/226 [02:13<00:00,  1.70it/s]


Epoch 0 
--- Loss 0.267 
---  Train Acc 0.911 
--- Test Acc 0.934 
--- Time 144.1s


训练轮数 [2/10]: 100%|██████████| 226/226 [02:15<00:00,  1.67it/s]


Epoch 1 
--- Loss 0.145 
---  Train Acc 0.947 
--- Test Acc 0.904 
--- Time 146.3s


训练轮数 [3/10]: 100%|██████████| 226/226 [02:15<00:00,  1.67it/s]


...

训练轮数 [4/10]: 100%|██████████| 226/226 [02:13<00:00,  1.69it/s]


Epoch 3 
--- Loss 0.104 
---  Train Acc 0.960 
--- Test Acc 0.957 
--- Time 144.3s


训练轮数 [5/10]: 100%|██████████| 226/226 [02:14<00:00,  1.68it/s]


Epoch 4 
--- Loss 0.096 
---  Train Acc 0.962 
--- Test Acc 0.955 
--- Time 145.6s


RuntimeError: Parent directory checkpoints does not exist.

In [17]:
pc = cv2.imread("NEU_Seg-main/NEU_Seg-main/images/training/000201.jpg")
print(np.array(pc))

[[[ 73  73  73]
  [ 74  74  74]
  [ 74  74  74]
  ...
  [106 106 106]
  [107 107 107]
  [110 110 110]]

 [[ 76  76  76]
  [ 76  76  76]
  [ 75  75  75]
  ...
  [107 107 107]
  [107 107 107]
  [104 104 104]]

 [[ 76  76  76]
  [ 75  75  75]
  [ 74  74  74]
  ...
  [104 104 104]
  [102 102 102]
  [ 98  98  98]]

 ...

 [[ 70  70  70]
  [ 70  70  70]
  [ 68  68  68]
  ...
  [ 91  91  91]
  [ 89  89  89]
  [ 92  92  92]]

 [[ 70  70  70]
  [ 71  71  71]
  [ 69  69  69]
  ...
  [ 86  86  86]
  [ 86  86  86]
  [ 90  90  90]]

 [[ 70  70  70]
  [ 71  71  71]
  [ 69  69  69]
  ...
  [ 90  90  90]
  [ 89  89  89]
  [ 91  91  91]]]
